# Data enrichment

In [223]:
import pandas as pd
import numpy as np
import re

### Function for extracting chunks of data with desired average_rating

In [224]:


def exract_df_ratings(chunksize,target_rows,rating):
    # Counter for total rows processed
    total_rows = 0
    chunks = []
    # Read the JSON file in chunks
    for chunk in pd.read_json("goodreads_books.json", lines=True, chunksize=chunksize):
        # Convert 'average_rating' to numeric (it is stored as a string)
        chunk['average_rating'] = pd.to_numeric(chunk['average_rating'], errors='coerce')

        
        # Replace empty strings with NA
        
        chunk = chunk.replace('', pd.NA)
        # Drop NAs
        chunk = chunk.dropna(subset=["isbn",'text_reviews_count','authors',
         'num_pages', 'isbn13','book_id', 'ratings_count','title','average_rating',"publication_year"])
         
         # Filter the chunk based on your condition
        
        

        filtered_chunk = chunk[chunk['average_rating'].round().astype("int64") == rating]
        
        
        
        chunks.append(filtered_chunk)
        
        # Update the counter for total rows processed
        total_rows += len(filtered_chunk)
        
        # Break out of the loop if  the target number of rows reached
        if total_rows >= target_rows:
            break

    # Concatenate all the filtered chunks into a single DataFrame
    final_df = pd.concat(chunks, axis=0)

    
    final_df = final_df.iloc[:target_rows]
    return final_df



 #### **1. Collecting  and merging the Dataframes**

In [225]:

df_rating_1=exract_df_ratings(20000,10000,1)

df_rating_2=exract_df_ratings(20000,10000,2)
df_rating_3=exract_df_ratings(20000,10000,3)
df_rating_5=exract_df_ratings(20000,10000,5)


In [226]:
df = pd.concat([ df_rating_2,df_rating_3,df_rating_5,df_rating_1]).reset_index(drop=True)

#### **2.Preproccesing and cleaning the Data**

In [227]:
df.columns

Index(['isbn', 'text_reviews_count', 'series', 'country_code', 'language_code',
       'popular_shelves', 'asin', 'is_ebook', 'average_rating', 'kindle_asin',
       'similar_books', 'description', 'format', 'link', 'authors',
       'publisher', 'num_pages', 'publication_day', 'isbn13',
       'publication_month', 'edition_information', 'publication_year', 'url',
       'image_url', 'book_id', 'ratings_count', 'work_id', 'title',
       'title_without_series'],
      dtype='object')

In [228]:
df.describe()

,average_rating,book_id
count,29437.000000,2.943700e+04
mean,3.379077,1.336023e+07
std,1.157223,1.011655e+07
min,1.000000,4.610000e+02
25%,2.500000,3.566391e+06
50%,3.310000,1.246190e+07
75%,4.610000,2.144644e+07
max,5.000000,3.646566e+07


In [229]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29437 entries, 0 to 29436
Data columns (total 29 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   isbn                  29437 non-null  object 
 1   text_reviews_count    29437 non-null  object 
 2   series                29437 non-null  object 
 3   country_code          29437 non-null  object 
 4   language_code         11032 non-null  object 
 5   popular_shelves       29437 non-null  object 
 6   asin                  0 non-null      object 
 7   is_ebook              29437 non-null  object 
 8   average_rating        29437 non-null  float64
 9   kindle_asin           10966 non-null  object 
 10  similar_books         29437 non-null  object 
 11  description           25935 non-null  object 
 12  format                28781 non-null  object 
 13  link                  29437 non-null  object 
 14  authors               29437 non-null  object 
 15  publisher          

#### **3. Selecting and formating relevent features** 

In [230]:
df=df[["isbn","language_code",'text_reviews_count','authors',
       'publisher', 'num_pages', 'isbn13','book_id', 'ratings_count','title','average_rating','publication_month','publication_year']]

In [231]:
df=df.rename(columns={"book_id":"bookID"})

In [232]:
df['average_rating'] = df['average_rating'].round().astype("int64") 
df['text_reviews_count'] = pd.to_numeric(df['text_reviews_count'], errors='coerce').astype("int64")
df['ratings_count'] = pd.to_numeric(df['ratings_count'], errors='coerce').astype("int64")
df['num_pages'] = pd.to_numeric(df['ratings_count']).astype("int64")


In [233]:

df['publication_year'] = pd.to_numeric(df['publication_year'], errors='coerce').astype("int64")

In [234]:
authors_df=pd.read_json("goodreads_book_authors.json", lines=True)
authors_df.rename(columns={authors_df.columns[3]:"name"},inplace=True)
authors_df['author_id'] = authors_df['author_id'].astype(str)

In [235]:
df[df.isbn=="9734661701"].authors

12574    [{'author_id': '1587258', 'role': 'coord.'}, {...
Name: authors, dtype: object

In [236]:
df['authors'] = df['authors'].astype(str)

In [237]:


def extract_author_ids_from_string(authors_str):
    return re.findall(r"'author_id': '(\d+)'", authors_str)

df['author_ids'] = df['authors'].apply(extract_author_ids_from_string)


In [238]:
print(df['author_ids'].head())

0      [35507, 4006336]
1              [556176]
2              [219818]
3    [3271740, 5413891]
4              [309210]
Name: author_ids, dtype: object


In [239]:
def get_author_names(author_ids):
    return ', '.join([author_map.get(str(aid), '') for aid in author_ids if str(aid) in author_map])


In [240]:
author_map = dict(zip(authors_df['author_id'].astype(str), authors_df['name']))

In [241]:
df['authors'] = df['author_ids'].apply(get_author_names)

In [242]:
df.drop(columns=[ 'author_ids'], inplace=True)

In [243]:
df.authors

0        Eric Ethan, Cameras in Action Staff
1                          Sandi Gelles-Cole
2                             Michael Powers
3                  Noel Slangen, Jan Withofs
4                                  Jean Ford
                        ...                 
29432                           Wendy Wagner
29433                            Miriam Pace
29434                            Lois Walker
29435                          Lynn M. Stone
29436                 Justin Dwayne Foxworth
Name: authors, Length: 29437, dtype: object

#### **4. Reviewing the Dataset**

In [244]:
def group(df,index,values,sort):
    table = df.copy().groupby(index).agg(values)
    return table.sort_values(by=sort,ascending=False).reset_index()

In [245]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29437 entries, 0 to 29436
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   isbn                29437 non-null  object
 1   language_code       11032 non-null  object
 2   text_reviews_count  29437 non-null  int64 
 3   authors             29437 non-null  object
 4   publisher           29046 non-null  object
 5   num_pages           29437 non-null  int64 
 6   isbn13              29437 non-null  object
 7   bookID              29437 non-null  int64 
 8   ratings_count       29437 non-null  int64 
 9   title               29437 non-null  object
 10  average_rating      29437 non-null  int64 
 11  publication_month   26880 non-null  object
 12  publication_year    29437 non-null  int64 
dtypes: int64(6), object(7)
memory usage: 2.9+ MB


In [246]:
df.language_code.value_counts()

eng      6526
ita       900
en-US     569
fre       521
ger       418
         ... 
enm         1
hye         1
tha         1
pes         1
cor         1
Name: language_code, Length: 64, dtype: int64

In [247]:
df.publisher.value_counts()

Createspace Independent Publishing Platform    995
Createspace                                    515
iUniverse                                      184
Authorhouse                                    174
Harlequin                                      169
                                              ... 
PM Press                                         1
Paranormal Association of Cold Cases             1
Cosmo Iannone                                    1
SU Strips                                        1
McGraw-Hill Science/Engineering/Math             1
Name: publisher, Length: 10533, dtype: int64

In [248]:
group(df,"average_rating",{"text_reviews_count":"mean","ratings_count":"mean",'num_pages':"mean","title":"count"},"title")

,average_rating,text_reviews_count,ratings_count,num_pages,title
0,3,23.144500,177.740600,177.740600,10000
1,5,13.815300,171.301100,171.301100,10000
2,2,3.001704,15.075228,15.075228,8215
3,1,1.391162,1.842881,1.842881,1222


#### **5. Saving the data in a CSV file**

In [249]:
df.to_csv("data_enrichement2.csv",index=False)